# 1424-Diagonal-Traverse-2

Given a 2D integer array `nums`, return _all elements of_ `nums` _in diagonal order as shown in the below images_.

**Example 1:**

![](https://assets.leetcode.com/uploads/2020/04/08/sample_1_1784.png)

**Input:** nums = \[\[1,2,3\],\[4,5,6\],\[7,8,9\]\]
**Output:** \[1,4,2,7,5,3,8,6,9\]

**Example 2:**

![](https://assets.leetcode.com/uploads/2020/04/08/sample_2_1784.png)

**Input:** nums = \[\[1,2,3,4,5\],\[6,7\],\[8\],\[9,10,11\],\[12,13,14,15,16\]\]
**Output:** \[1,6,2,8,7,3,9,4,12,10,5,13,11,14,15,16\]

**Constraints:**

*   `1 <= nums.length <= 105`
*   `1 <= nums[i].length <= 105`
*   `1 <= sum(nums[i].length) <= 105`
*   `1 <= nums[i][j] <= 105`

## Info

- Link: https://leetcode.com/problems/diagonal-traverse-ii
- Date: 2023/11/22
- Difficulty: Medium

## First Try

Q: Will it be a $n*n$ matrix? Or can it be a $n*m$ matrix?

Let's assume it is a $n*n$ matrix.

First diagonal is [0,0], then [[1,0], [0,1]], then [[2,0], [1,1], [0,2]]...

The sum of two coordinate is equal to the index of this turn.

so:

```python
listFinal = []
for turn in range(0, len(nums)):
    for y in range(0, turn):
        x = turn - y
        listFinal.append(listNew[x][y])
```

The upper half matrix is done. Now for the lower half.

This is not elegant. Use a single loop to handle all situations.

Add a method to check if it is out of range.



```python
for turn in range(0, (len(nums)-1)*2):
    for y in range(0, turn):
        x = turn - y
        if x<len(nums) and y<len(nums):
            listFinal.append(listNew[x][y])
```



We can add `-1` to empty cells then delete it so we don't need to check whether there exists elements at each cell.

```python
numsLen = len(nums)
listNew = []
for list in nums:
    innerList = [None] * numsLen
    i = 0
    for num in list:
        innerList[i] = list[i]
        i += 1
    for j in range(i, numsLen):
        innerList[j] = -1
    listNew.append(innerList)

```

So the final code is:

In [3]:
class Solution(object):
    def findDiagonalOrder(self, nums):
        # use a new list to represent a full matrix
        # store -1 to empty cells
        numsLen = len(nums)
        listNew = []
        for list in nums:
            innerList = [None] * numsLen
            i = 0
            for num in list:
                innerList[i] = num
                i += 1
            for j in range(i, numsLen):
                innerList[j] = -1
            listNew.append(innerList)
        
        # add them to a new list by order
        listFinal = []
        for turn in range(0, (len(nums)-1)*2):
            for y in range(0, turn):
                x = turn - y
                if x<len(nums) and y<len(nums):
                    listFinal.append(listNew[x][y])
        
        # delete all -1
        while -1 in listFinal:
            listFinal.remove(-1)

Error: has no output

Why: forgot to return `listFinal`

Error: missing some elements



## First Submit

Failed: It can be a $n*m$ matrix

Solution: refine the numsLen method, use a dim to store the biggest value

After some refactor, my code is:

In [4]:
class Solution(object):
    def findDiagonalOrder(self, nums):
        # use a new list to represent a full matrix
        # store -1 to empty cells

        # decide the dimention of new matrix
        dim = len(nums)
        for list in nums:
            if len(list) > dim:
                dim = len(list)

        initialInnerList = [-1] * dim
        listNew = [initialInnerList] * dim
        
        i = 0
        for list in nums:
            j = 0
            for num in list:
                listNew[i][j] = num
                j += 1
            i+= 1
        
        # add them to a new list by order
        listFinal = []
        for turn in range(0, 2*dim-1):
            for y in range(0, turn+1):
                x = turn - y
                if x<dim and y<dim:
                    listFinal.append(listNew[x][y])
        
        # delete all -1
        while -1 in listFinal:
            listFinal.remove(-1)

        return listFinal

and it does not pass the test now. The problem looks like: [7,7,8,7,8,9,8,9,9] this is my output.

Why: 

> The issue might be with how you're initializing `listNew`. If you're initializing it with something like `listNew = [[0]*cols]*rows`, it will create a list of lists where all the inner lists are the same object. So, when you modify one row, all rows will be modified.
> Instead, you should initialize `listNew` with a list comprehension to ensure that each row is a separate object.

```python
listNew = [[0 for _ in range(cols)] for _ in range(rows)]
```

In [5]:
class Solution(object):
    def findDiagonalOrder(self, nums):
        # use a new list to represent a full matrix
        # store -1 to empty cells

        # decide the dimention of new matrix
        dim = len(nums)
        for list in nums:
            if len(list) > dim:
                dim = len(list)

        listNew = [[-1 for _ in range(dim)] for _ in range(dim)]

        i = 0
        for list in nums:
            j = 0
            for num in list:
                listNew[i][j] = num
                j += 1
            i+= 1
        
        # add them to a new list by order
        listFinal = []
        for turn in range(0, 2*dim-1):
            for y in range(0, turn+1):
                x = turn - y
                if x<dim and y<dim:
                    listFinal.append(listNew[x][y])
        
        # delete all -1
        while -1 in listFinal:
            listFinal.remove(-1)

        return listFinal

## Second Submit

Works, but Time Limit Exceeded.

Hint: Store them in tuples (sum, row, val), sort them, and then regroup the answer.

### What is tuple in python?

Ref: https://www.youtube.com/watch?v=NI26dqhs2Rk

- Tuple is a smaller data type than List
- Tuple cannot be changed, which is also called immutable. Once it is created, the data is graved on stone.
- 

In [6]:
class Solution(object):
    def findDiagonalOrder(self, nums):
        # use tuples to store data
        data = []
        i = 0
        for list in nums:
            j = 0
            for num in list:
                data.append((i+j, j, num)) # sum, column, num
                j += 1
            i += 1
        
        # sort tuples, first by sum, then by column
        data.sort(key=lambda x: (x[0], x[1]))

        # output
        final = []
        for num in data:
            final.append(num[2])
        
        return final
        

                
            

Passed!

## Overview

- Use Python's unique data type, and use functions python provides. This can save time and memory than writing your own method.
- Turn problems into existing problems. Like in this problem, thinking in matrix can lead to much more redundant work, while turning it into a two-times sort problem with priority is much easier to solve.

In [7]:
# Generated by GitHub copilot

class Solution(object):
    def findDiagonalOrder(self, nums):
        """
        :type nums: List[List[int]]
        :rtype: List[int]
        """
        res = []
        m = len(nums)
        for i in range(m):
            for j in range(len(nums[i])):
                if len(res) <= i + j:
                    res.append([])
                res[i+j].append(nums[i][j])
        ans = []
        for i in range(len(res)):
            ans += res[i][::-1]
        return ans